# Loan Default Classification System

en README..

**Context:**

**User:** Banks, SOFOM, SOFIPO, any institution that gives loans.

As a bank i want to know in advancement if a client will likely default their next payment so that I can act in advance and prevent it or prepare for it. 

**Objective:** 
- Dado un cliente que tiene un crédito activo, quisiera saber si pagará o no, con respecto a comportamientos en algunos de sus features. 
- ¿Cuales son los features que indican que el cliente fallará? 
 

----
**Tech Stack:** Python, Pandas, Numpy, Scikit-learn, Matplotlib, Gradio

**Description:**

In [1]:
# Install packages
!pip install kagglehub==0.4.0

In [2]:
# Import libraries. 
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Download latest version
path = kagglehub.dataset_download("yasserh/loan-default-dataset")

df = pd.read_csv(f"{path}/Loan_Default.csv")

In [ ]:
# Dataset exploration
df.info()

cols_w_missing_v = df.columns[df.isnull().any()]
print()
print(f"Columns with missing values: {len(cols_w_missing_v)}{"\n"}{cols_w_missing_v}")

In [ ]:
# Counting missing values per column
def missing_data(dataframe): 
    return dataframe.isnull().sum()

missing_data(df)

In [ ]:
# check % of missing values. If a column has more than 50% of missing values, it will be eliminated. 
missing_values = pd.DataFrame(missing_data(df), columns=["missing_values"])
missing_values["missing_pct"] = missing_values/len(df)

cols_to_drop = missing_values[missing_values["missing_pct"] > 0.5].index
cols_to_drop

# No columns above 50% with missing data. 

In [ ]:
# check for records that have more than 50% in nan and eliminate them if any. 
threshold = 0.5

rows_to_drop = df.isna().mean(axis=1) > threshold
df.loc[rows_to_drop==True].count().sum() # count for any that was above 0.5. 

# No records to eliminate

In [ ]:
# Check for duplicates to minimize missing values.
df.duplicated().sum()

# No duplicates

In [ ]:
# Obtain non numerical columns and extract unique values to get more insights for missing values treatment. 
non_num_cols = df.select_dtypes(exclude="number").columns
print("Non-numerical columns:", "\n", len(non_num_cols))

# Save the columns that have 2 values and a nan(if applicable). This will help transform them into binary values later. 
binary_values = {}

# Explore type of information in each column. For that let's print the unique values of each. 
for record in non_num_cols: 
    
    unique_values = df[record].unique()
    print("\n", record, ": ", unique_values)
    
    if (len(unique_values) == 3 and np.any(pd.isna(unique_values))) or len(unique_values) == 2:
        binary_values[record] = unique_values
    

print("\n", f'total of columns with binary values: {len(binary_values)}', "\n", binary_values)

The columns with missing values are: 

|Coumns|Missing values|
|------|-------------|
|loan_limit                    |3344| 
|approv_in_adv                  |908|
|loan_purpose                   |134|
|rate_of_interest             |36439|
|Interest_rate_spread         |36639|
|Upfront_charges              |39642|
|term                            |41|
|Neg_ammortization              |121|
|property_value               |15098|
|income                        |9150|
|age                            |200|
|submission_of_application      |200|
|LTV                          |15098|
|dtir1                        |24121|

Some highlights: 
- In banking the following are gold features, thus the nan values can not be eliminated, since they also give certain type of information. Based on research:
  
  |Feature|Meaning|
  |------|-------|
  |"rate_of_interest"|not given one|
  |"Interest_rate_spread"|derived from the rate_of_interest. Missing values to calculate|
  |"Upfront_charges"|client with different product or not evaluated|
  |"dtir1"| debt-to-income not calculated|
  
- "property_value" means that the appraisal was not made.
- LTV derivates from property_value. LTV is calculated as *loan_amount/property_value*, which means that if we don't have the property value, we can't calculate it. In banking LTV is highly interpretable for determining default. A high LTV means a higher default risk. The missing values, will be kept. 
- "income" means it wasn't declared
- "submission_of_application" can not be determined by any other feature. The feature has 2 values, *to_inst* and *not_inst*. *not_inst* means that the solicitation was either incomplete, it was a pre-application, or the lead was not converted. Thus since the loan never started, this records will be deleted.
- Features that could be imputable are: "loan_limit", "approv_in_adv", "loan_purpose", "term", "Neg_ammortization", "age", "submission_of_application"

In [ ]:
# Delete records that were not a loan
df_loans = df.copy()
df_loans = df_loans[df_loans["submission_of_application"] != "not_inst"]

# check for missing data reduction
print(f"Columns with missing values: {(missing_data(df_loans) > 0).sum()}{"\n"}")
# no reduction of columns with missing data. 

missing_data(df_loans)
# Reduction of number of missing data

In [ ]:
# Since we want to try out different models (logistic regression and XGBoost), 
# it's important to create different datasets that can be handled by them. 
# Logistic regression = doesn't tolerate missing values and outliers
# XGBoost = tolearates missing values and outliers

df_na = df_loans.copy() # DF with native NA. No changes needed 

# Create the missing flags for the golden features and put the median in the nan values. 
df_no_na = df_loans.copy() # DF with no NA, but with missing flags and imputed values. 
missing_flags_cols = ["rate_of_interest", "Interest_rate_spread", 
                      "Upfront_charges", "dtir1", "property_value", 
                      "LTV", "income"]

for col in missing_flags_cols: 
    df_na[col+"_missing"] = df_na[col].isna().astype(int)
    df_no_na[col+"_missing"] = df_no_na[col].isna().astype(int)
    df_no_na[col] = df_no_na[col].fillna(df_no_na[col].median())

# Insert the median in numerical data. 
df_no_na["term"] = df_no_na[col].fillna(df_no_na[col].median())

# For categorical columns, fillna with an extra category called "Missing"
missing_val_cat_cols = ["loan_limit", "approv_in_adv", "loan_purpose", 
                        "Neg_ammortization", "submission_of_application", "age"]
for col in missing_val_cat_cols:
    df_no_na[col] = df_no_na[col].fillna("Missing")

# check for missing values again in the df_no_na df. 
missing_data(df_no_na)

In [ ]:
# Check for outliers
# Extract numeric_columns
numeric_cols = df_no_na.select_dtypes(exclude="object")

# Eliminate non valuable columns (such as the ones with "_missing")
numeric_cols = list(numeric_cols.columns[~numeric_cols.columns.str.contains("missing")])
del numeric_cols[:2]

# Remove Status
numeric_cols.remove("Status")
numeric_cols

# Calculate IQR
def iqr_outliers(feature):
    q1 = feature.quantile(0.25)
    q3 = feature.quantile(0.75)
    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper

# detect the outliers per column 
outlier_summary = {}
for col in numeric_cols: 
    f = df_no_na[col].dropna()
    lower, upper = iqr_outliers(f)

    n_outliers = ((f < lower) | (f > upper)).sum()
    outlier_summary[col] = {
        "lower": lower, 
        "upper": upper, 
        "n_outliers": n_outliers,
        "pct_outliers": round((n_outliers/len(f)*100), 2)
    }

possible_errors = []
rare_values = []
long_tales = []
subsamples = []

for key in outlier_summary.keys(): 
    if (outlier_summary[key]["pct_outliers"] < 0.5) and (outlier_summary[key]["pct_outliers"] != 0):
        possible_errors.append(key)
    elif (outlier_summary[key]["pct_outliers"] > 0.5) and (outlier_summary[key]["pct_outliers"] < 2): 
        rare_values.append(key)
    elif (outlier_summary[key]["pct_outliers"] > 2) and (outlier_summary[key]["pct_outliers"] < 10):
        long_tales.append(key)
    else:
        subsamples.append(key)

print(f"Possible features with errors {possible_errors}")
print(f"rare values {rare_values}")
print(f"long tale {long_tales}")
print(f"subsamples {subsamples}")

The Long tales will be the ones to watch out for a possible transformation. 

In [ ]:
# plot outliers
def plot_outliers(feature, v_df):
    for col in feature:
        plt.figure(figsize=(6,2))
        sns.boxplot(x=v_df[col])
        plt.title(col)
        plt.show()

plot_outliers(long_tales, df_no_na)

The outliers shown above are logical, although in some circunstances like LTV, income and term present some extreme values, they represent situations that can happen in real life with some exceptional clients. Thus such cases will not be eliminated, but transformed to: 
- Reduce asymmetry
- smooth long tales
- reduce extreme values impacts
- improve model's stability

However LTV, dtir1 and rate_of_interest, will be winsorized instead of transformed due to the fact that these metrics are ratios. If these are transformed the linear relationship between them and the risk would be broken, eliminating the insights of risk behavior.

In [ ]:
tales_to_transform = ["loan_amount", "property_value", "income"]
tales_to_winsor = ["Interest_rate_spread", "LTV", "dtir1"]

for col in tales_to_transform:
    df_no_na[col] = np.log1p(df_no_na[col])

# plot to see the new ranges of outliers
plot_outliers(long_tales, df_no_na)

In [ ]:
# Obtain correlations between features vs target. 
# add the status column again. 
# numeric_cols.append("Status") # Run this line 1 time. 

# check for numerical correlations
corr = df_no_na[numeric_cols].corr(method="spearman")

# plot correlations
plt.figure(figsize=(7,5))
sns.heatmap(corr)
plt.title("Numerical correlations")
plt.show()

There are some important correlations here between variables, but none strong enough with the target. Some of the most highliting correlated variables are: 

|Features|Interpretation|
|--------|--------------|
|income and term| Very Strong correlation| 
|property_value and loan_amount| Strong correlation|
|term and loan_amount| Moderate correlation|
|income and loan_amount| Moderate correlation|

In [ ]:
# obtain high correlations pairs
corr = corr.abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)

high_corr_pairs = high_corr_pairs[high_corr_pairs[0] > 0.7]
high_corr_pairs

In [ ]:
# Check whether they influence in the same proportion to the target. 
unique_values = []
for col in high_corr_pairs.columns[0:2]:
    unique_values.extend(high_corr_pairs[col].unique())

for value in unique_values: 
    print(df_no_na.groupby(pd.qcut(df_no_na[value], 4, duplicates="drop"), observed=True)["Status"].mean())

Term and income show the same curves, which means a clear redundancy. Thus one of both, must be eliminated. To decide which one, let's check the roc_auc score.

In [ ]:
# Import libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Define x and y
x = df_no_na[numeric_cols[:-1]]
y = df_no_na[numeric_cols[-1]]

# Calculate AUC for 1 feature
def univariate_auc(dfo, feature, target="Status"):
    x = df_no_na[[feature]]
    y = df_no_na[target]

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y
    )

    model = LogisticRegression(max_iter=1000)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)

    return auc

auc_results = []

#for col in unique_values["term", "income"]:
#    print(col)

unique_values["term"]

In [ ]:
# Check for categorical correlacion
cat_cols = df_no_na.select_dtypes(exclude="number")
cat_cols.columns

for col in cat_cols: 
    print(df_no_na.groupby(col)["Status"].mean())
    
# Obtener correlaciones solo que divide entre categorias y numericas 

# obten correlacion num-num y cat vs target. 
# codea el one-hot encoding 

# entrena 

# evalúa

# git! 

In [ ]:
# # Transform features with 2 categories to binary. 
# vals = {}
# for key in binary_values: 
#     values = binary_values[key]
    
#     vals[key] = {}
#     val = 0
    
#     for value in values:
        
#         if pd.isna(value):
#             continue
            
#         vals[key][value] = val
#         val += 1 

# vals

In [ ]:
# df_clean = df.copy()
# for col, mapping in vals.items():
#     df_clean[col] = df[col].map(mapping)

# non_num_cols = df_clean.select_dtypes(exclude="number").columns
# print("Non-numerical columns:", "\n", len(non_num_cols))

# non_num_cols

In [ ]:
# df_clean.head(2)

In [ ]:
# check which variables predict the status. 
# first numerical 
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.drop(["Status", "ID", "year"])

num_cols

differences = []

for col in num_cols:
    
    diff = abs(df[df==1][col].notna().median() - df[df==0][col].notna().median())
    differences.append((col, diff))

differences

# categorical 

In [ ]:
# Which variables distinguish clients that pay vs the ones that don't pay. 


In [ ]:
# check if the target is balanced. 0 = pays, 1 = doesn't pay 
target_counts = df_clean["Status"].value_counts()

fig, ax = plt.subplots()
x = ["pays[0]", "Doesn't pay[1]"]
y = target_counts

ax.bar(x, y, color=['limegreen', 'firebrick'])
ax.set_title("Number of clients with Payment default")

plt.show()